<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/VGG16_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.utils import np_utils
from PIL import Image
import pandas as pd
import os
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_train_data():
    X_train = np.load('/content/drive/My Drive/IEEE/NPY/X_train.npy')
    Y_train = np.load('/content/drive/My Drive/IEEE/NPY/Y_train.npy')
    return X_train, Y_train

In [ ]:
def load_test_data():
    X_test = np.load('/content/drive/My Drive/IEEE/NPY/X_test.npy')
    Y_test = np.load('/content/drive/My Drive/IEEE/NPY/Y_test.npy')
    return X_test, Y_test

In [ ]:
(X_train, y_train) = load_train_data()
(X_test, y_test) = load_test_data()

print(y_train[:1])
# data pre-processing
X_train = X_train.reshape(-1, 1, 224, 224)/255.
X_test = X_test.reshape(-1, 1, 224, 224)/255.
Y_train = np_utils.to_categorical(y_train, num_classes=None)
Y_test = np_utils.to_categorical(y_test, num_classes=None)
print(Y_train[:1])

[[0.]]
[[1. 0.]]


In [ ]:
# Another way to build your CNN
_input = Input(shape=(1,224,224))

#Block 1
x = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(_input)
x = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = MaxPooling2D(pool_size=2, strides=2, data_format='channels_first')(x)

#Block 2
x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = MaxPooling2D(pool_size=2, strides=2, data_format='channels_first')(x)

#Block 3
x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = MaxPooling2D(pool_size=2, strides=2, data_format='channels_first')(x)

#Block 4
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = MaxPooling2D(pool_size=2, strides=2, data_format='channels_first')(x)

#Block 5
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', data_format='channels_first')(x)
x = MaxPooling2D(pool_size=2, strides=2, data_format='channels_first')(x)

pooling_layer2 = BatchNormalization()(x)
flatten = Flatten()(pooling_layer2)

dense1 = Dense(4096, activation="relu")(flatten)
dense2 = Dense(4096, activation="relu")(dense1)

#output FC
output = Dense(2, activation="sigmoid")(dense2)

model_dense2_output = Model(inputs=_input, outputs=dense2)
model = Model(inputs=_input, outputs=output)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 224, 224)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 224, 224)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 112, 112)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 112, 112)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 112, 112)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 56, 56)      

In [ ]:
adam = Adam(lr=0.01)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print('Training ------------')
# Another way to train the model
history = model.fit(X_train, Y_train, epochs=500, batch_size=8)

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(X_test, Y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

path_save_train = '/content/drive/My Drive/IEEE/Feature/train_VGG16.csv'
path_save_test = '/content/drive/My Drive/IEEE/Feature/test_VGG16.csv'

X_train_xg = model_dense2_output.predict(X_train)
X_test_xg = model_dense2_output.predict(X_test)
print(np.array(X_train_xg).shape)
print(np.array(y_train).shape)
print(np.array(X_test_xg).shape)
print(np.array(Y_test).shape)

#Quantidade de dados de treinamento
z = np.concatenate([np.array(X_train_xg).reshape(566,4096),np.array(y_train).reshape(566,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv(path_save_train,index=False)

#Quantidade de dados de teste
z = np.concatenate([np.array(X_test_xg).reshape(142,4096),np.array(y_test).reshape(142,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv(path_save_test,index=False)

Training ------------
Epoch 1/500
71/71 [==============================] - 11s 158ms/step - loss: 8.8662 - accuracy: 0.4452
Epoch 2/500
71/71 [==============================] - 10s 145ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 3/500
71/71 [==============================] - 10s 147ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 4/500
71/71 [==============================] - 11s 149ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 5/500
71/71 [==============================] - 11s 151ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 6/500
71/71 [==============================] - 11s 153ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 7/500
71/71 [==============================] - 11s 156ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 8/500
71/71 [==============================] - 11s 159ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 9/500
71/71 [==============================] - 11s 160ms/step - loss: 8.9703 - accuracy: 0.4435
Epoch 10/500
71/71 [==============================] - 11s 15